需要给已经注释的uniprot蛋白添加以下的信息
[1] "ENS_id"          "ORF_id_trans"    "ORF_id_seq"      "Seq"             "Scodon"          "Type"            "Strand"         
[8] "Chr"             "Start"           "End"             "Length"          "Gene_name"       "Transcript_type" "Gene_type"      
[15] "CDS_start"       "CDS_end"  "Unique_psm_n"	"Unique_peptide_n" "Ribo_evidence" "ENS_p_id"

In [1]:
source("/home/user/data2/lit/bin/lit_utils.R")
source("/home/user/data3/lit/project/sORFs/sORFs.utils.R")
lib_text()

In [2]:
trans_mul_feature_path <- "/home/user/data2/lit/project/ZNF271/data/annotation/Ensembl_106_Gencode_v41_Human_Transcript_stable_ID_version_Gene_stable_ID_version_Gene_name_Transcript_type_gene_type.txt"
anno_sep_gpe="../processed/uniprot_id_ens_id_PE_type/uniprot.human.sep.15.gpe"
anno_sep_tab="../processed/uniprot_id_ens_id_PE_type/tmp/uniprot.human.sep.rmdup.tab"
anno_sep_type_path="../processed/uniprot_id_ens_id_PE_type/uniprot.human.sep.id.type.txt"
uniprot_id_ensId="../processed/uniprot_id_ens_id_PE_type/perfect_match_to_ensId.uniq.txt"

In [3]:
fread_c(anno_sep_gpe) -> uniprot.human.sep.15.gpe
head(uniprot.human.sep.15.gpe,1)
# uniprot id和seq
fread(anno_sep_tab,sep='\t',header = F,data.table = F) -> uniprot_id_seq
colnames(uniprot_id_seq) <- c("Uniprot_id","Seq")
head(uniprot_id_seq,1)
# uniprot id和转录本id
fread(uniprot_id_ensId,sep='\t',header = F,data.table = F) -> uniprot_id_ensembl_t_id_l
colnames(uniprot_id_ensembl_t_id_l) <- c("Uniprot_id","ENS_id","Length",".",".")
head(uniprot_id_ensembl_t_id_l,1)
fread(anno_sep_type_path,header = F,data.table = F,sep='\t')->anno_sep_type
colnames(anno_sep_type)=c("Uniprot_id","PE_type")
head(anno_sep_type,1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,ENST00000344843.12,chr1,-,1401908,1407293,1402082,1407217,4,"1401908,1405808,1406908,1407130,","1402256,1405886,1407019,1407293,",0,MRPL20,cmpl,cmpl,"0,0,0,0,"


,Uniprot_id,Seq
,<chr>,<chr>
1,sp|A0A0B4J2F0|PIOS1_HUMAN,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLVQESEEKKS


,Uniprot_id,ENS_id,Length,.,.,NA
,<chr>,<chr>,<int>,<int>,<int>,<lgl>
1,sp|A0A024R1R8|TMA7B_HUMAN,ENST00000424496.3,64,1,64,NA


,Uniprot_id,PE_type
,<chr>,<chr>
1,sp|A0A0B4J2F0|PIOS1_HUMAN,PE_1


In [7]:
merge(uniprot_id_seq,uniprot_id_ensembl_t_id_l[,c("Uniprot_id","ENS_id","Length")],by="Uniprot_id") %>% 
  merge(uniprot.human.sep.15.gpe,by.x="ENS_id",by.y="V1") -> m_1
m_1 %>% dplyr::rename(Chr=V2,Strand=V3,Start=V6,End=V7) -> m_2
m_2[,!grepl("^V",colnames(m_2))] -> m_2
mutate(m_2,
       ORF_id_trans=paste0(ENS_id,Strand,Chr,":",Start,"-",End),
       ORF_id_seq=paste0(Strand,Chr,":",Start,"-",End,":",Seq),
       Scodon="ATG",
       Type="Canonical",
       ORF_type_1="Canonical",
       ORF_type_2="Canonical",
       ORF_type_3="Canonical") -> m_2
# 加载基因名、转录本类型等信息并整合
trans_mul_feature <- fread(trans_mul_feature_path)
colnames(trans_mul_feature) <- c("Gene_ID", "Transcript_ID", "Transcript_type", "Gene_type", "Gene_name")
add_info_1 <- function(df){
  merge(df, trans_mul_feature,
        by.x = "ENS_id", by.y = "Transcript_ID") -> df
  df %>% as.data.frame() -> df
  return(df)
}
add_info_1(m_2) -> m_3
# 添加CDS_start以及CDS_end信息
uniprot.human.sep.15.gpe -> gpe
gpe[, c("V1", "V6", "V7")] -> gpe_selected
colnames(gpe_selected) <- c("ENS_id", "CDS_start", "CDS_end")
merge(m_3, gpe_selected) -> m_4

In [8]:
merge(m_4,anno_sep_type,by="Uniprot_id") -> m_5
head(m_5,1)
colnames(m_5)
nrow(m_5)

,Uniprot_id,ENS_id,Seq,Length,Chr,Strand,Start,End,ORF_id_trans,ORF_id_seq,⋯,ORF_type_1,ORF_type_2,ORF_type_3,Gene_ID,Transcript_type,Gene_type,Gene_name,CDS_start,CDS_end,PE_type
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
1,sp|A0A024R1R8|TMA7B_HUMAN,ENST00000424496.3,MSSHEGGKKKALKQPKKQAKEMDEEEKAFKQKQKEEQKKLEVLKAKVVGKGPLATGGIKKSGKK,64,chr22,+,39964374,39964569,ENST00000424496.3+chr22:39964374-39964569,+chr22:39964374-39964569:MSSHEGGKKKALKQPKKQAKEMDEEEKAFKQKQKEEQKKLEVLKAKVVGKGPLATGGIKKSGKK,⋯,Canonical,Canonical,Canonical,ENSG00000225528.4,protein_coding,protein_coding,ENSG00000225528,39964374,39964569,PE_3


[1] "Uniprot_id"      "ENS_id"          "Seq"             "Length"         
 [5] "Chr"             "Strand"          "Start"           "End"            
 [9] "ORF_id_trans"    "ORF_id_seq"      "Scodon"          "Type"           
[13] "ORF_type_1"      "ORF_type_2"      "ORF_type_3"      "Gene_ID"        
[17] "Transcript_type" "Gene_type"       "Gene_name"       "CDS_start"      
[21] "CDS_end"         "PE_type"

[1] 3243

In [9]:
fwrite_c(m_5,"../processed/tab_info/cano.sep.tab.info.txt")